# Executar um script de treinamento com SDK do Python

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    #check if given credential can get token successfully
    credential.get_token('https://management.azure.com/.default')
except Exception as ex:
    #fall back to InteractiveBrowserCredential if DefaultAzureCredential fails
    credential = InteractiveBrowserCredential()

In [ ]:
# Get a handle to the ML workspace
ml_client = MLClient.from_config(credential=credential)

In [ ]:
%%writefile src/sorvetes-training.py
#import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

#load the sorvetes dataset
print("Loading data...")
sorvetes = pd.read_csv("vendas_sorvetes.csv")

#separate features and labels

X, y = sorvetes[["Temperatura"]].values, sorvetes["Vendas"].values

#split data into training and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#set regularization hyperparameter
reg = 0.01

#train a logistic regression model
print("Training a logistic regression model with regularization parameter of", reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

#calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print("Accuracy:", acc)

#calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test, y_scores[:,1])
print("AUC: " + str(auc))

In [ ]:
from azure.ai.ml import command

#configure job
job = command(
    code="./src",
    command="python sorvetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="sorvetes-pythonv2-train",
    experiment_name="sorvetes-training"
)

#submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)